In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from scipy.stats import binned_statistic_2d

from matplotlib import pyplot as plt
%matplotlib inline

In [2]:
period_full = pd.date_range(datetime(2016,1,1), datetime(2016,6,30,23), freq='H')
period_train = pd.date_range(datetime(2016,1,1), datetime(2016,4,30,17), freq='H')
period_train2 = pd.date_range(datetime(2016,1,1), datetime(2016,5,31,17), freq='H')
period_validation = pd.date_range(datetime(2016,4,30,23), datetime(2016,5,31,17), freq='H')
period_test = pd.date_range(datetime(2016,5,31,23), datetime(2016,6,30,17), freq='H')

In [3]:
# праздники
holidays = pd.read_csv('usa holidays.csv', header=None, parse_dates=[0])
holidays.columns = ['date']
holidays = holidays.set_index('date')
holidays['holiday'] = 1

# регионы
regions = [int(reg) for reg in pd.read_csv('regions.csv').columns.values]
len(regions)

102

In [4]:
# координаты Нью-Йорка
nyLongFrom, nyLongTo = -74.25559, -73.70001
nyLatFrom, nyLatTo = 40.49612, 40.91553



# очищаем данные
def clear_data(data):
    # в сырых данных реестр колонок иногда меняется, приведем все имена колонок в нижний реестр
    data.columns = [str(col).lower() for col in data.columns]
    
    to_drop_indicies = data[
        (data['pickup_latitude'] < nyLatFrom) |
        (data['pickup_latitude'] > nyLatTo) |
        (data['pickup_longitude'] < nyLongFrom) |
        (data['pickup_longitude'] > nyLongTo) |
        (data['passenger_count'] == 0) |
        (data['trip_distance'] == 0) |
        (data['tpep_dropoff_datetime'] <= data['tpep_pickup_datetime']) ].index
    data = data.drop(to_drop_indicies, axis=0)
    return data



# добавляем регионы начала и окончания пути
def add_region(data):
    _, _, _, binnum_pickup = binned_statistic_2d(data['pickup_longitude'].to_numpy(),
                                                 data['pickup_latitude'].to_numpy(),
                                                 values = None, statistic = 'count',
                                                 bins = [50, 50],
                                                 range = [[nyLongFrom, nyLongTo], [nyLatFrom, nyLatTo]],
                                                 expand_binnumbers = True)
    pickup_region = (binnum_pickup[0] - 1) * 50 + binnum_pickup[1]
    
    _, _, _, binnum_dropoff = binned_statistic_2d(data['dropoff_longitude'].to_numpy(),
                                                 data['dropoff_latitude'].to_numpy(),
                                                 values = None, statistic = 'count',
                                                 bins = [50, 50],
                                                 range = [[nyLongFrom, nyLongTo], [nyLatFrom, nyLatTo]],
                                                 expand_binnumbers = True)
    dropoff_region = (binnum_dropoff[0] - 1) * 50 + binnum_dropoff[1]
    
    data['pickup_region'] = pickup_region
    data['dropoff_region'] = dropoff_region
    
    return data



# доля каждого категориального признака в отдельный признак
def feature_proportion(data, feature, new_colName):
    feature_types = np.sort(data[feature].unique())
    feature_dic = dict(zip(feature_types, ['{}_{}'.format(new_colName, i) for i in feature_types]))
    
    # группируем по региону, времени и <признаку>
    groupped = data.groupby(by=['pickup_region', 'tpep_pickup_datetime', feature],
                            as_index=False)[feature].agg(['count'])
    
    groupped = groupped.reset_index([feature])
    
    # меняем '1' на 'feature_1'
    groupped[feature] = groupped[feature].map(feature_dic)

    # кол-во поездок в каждой паре регион-час
    counts = groupped.groupby(by=['pickup_region', 'tpep_pickup_datetime'])['count'].sum()

    # создаем сводную
    pivot = groupped.pivot_table(values='count', index=['pickup_region', 'tpep_pickup_datetime'],
                                 columns=feature).fillna(0)
    # делим на общее кол-во поездок в данный час - получаем долю типа оплаты
    for col in pivot.columns[:-1]:
        pivot[col] = pivot[col] / counts
    
    return pivot


# преобразование и добавление признаков
def transform_data(data_raw):    
    # добавляем регионы посадки и высадки
    data_raw = add_region(data_raw)
    
    # добавляем длительность поездки
    data_raw['duration_sec'] = data_raw['tpep_dropoff_datetime'] - data_raw['tpep_pickup_datetime']
    data_raw['duration_sec'] = data_raw['duration_sec'].dt.total_seconds()
    
    # обрезаем время начала и окончания поездки до часов
    data_raw['tpep_pickup_datetime'] = data_raw['tpep_pickup_datetime'].dt.floor('H')
    data_raw['tpep_dropoff_datetime'] = data_raw['tpep_dropoff_datetime'].dt.floor('H')
    
    # кол-во поездок в паре регион-час
    data = data_raw.groupby(by=['pickup_region', 'tpep_pickup_datetime']).size().to_frame(name='count')
    
    # группируем признаки средних
    columns_mean = ['duration_sec', 'passenger_count', 'trip_distance', 'total_amount']
    data_means = data_raw.groupby(by=['tpep_pickup_datetime', 'pickup_region'])[columns_mean].mean()
    data = data.join(data_means)
    
    # доли поездок по тарифам каждого типа
    data_rate = feature_proportion(data_raw, 'ratecodeid', 'rate')
    data = data.join(data_rate)
    
    # доли поездок по способам оплаты каждого типа
    data_payment = feature_proportion(data_raw, 'payment_type', 'payment')
    data = data.join(data_payment)
    
    # доли поездок по провайдерам каждого типа
    data_provider = feature_proportion(data_raw, 'store_and_fwd_flag', 'provider')
    data = data.join(data_provider)
    
    return data_raw, data



# основной пайплайн обработки данных
def load_transform_pipeline(file_list):
    result_data, result_data_raw = None, None
    
    for i, file_path in enumerate(file_list):
        print('file {} of {} processing'.format(i+1, len(file_list)))
        
        data = pd.read_csv(file_path, parse_dates=['tpep_pickup_datetime', 'tpep_dropoff_datetime'])
        
        data = clear_data(data)
        
        data_raw, data = transform_data(data)
        
        if result_data is None:
            result_data_raw = data_raw
            result_data = data
        else:
            result_data_raw = pd.concat([result_data_raw, data_raw])
            result_data = pd.concat([result_data, data])
        
    result_data.reset_index('pickup_region', inplace=True)
    return result_data_raw, result_data



# добавлени признаков: год, меся, день, день недели, праздники,
# кол-во поездок n-часов назад, предыдущие n-часов в текущем и соседних регионах
def time_features(data, hours_back=[1, 6, 24, 48], rolling_back=[6, 24]):   
    data['year'] = data.index.year
    data['month'] = data.index.month
    data['day'] = data.index.day
    data['weekday'] = data.index.weekday
    
    for i in hours_back:
        data['t-%d'%i] = data.iloc[:, 0].shift(i)
    
    for i in rolling_back:
        data['prev_%dh'%i] = data.iloc[:, 0].rolling(i, i, closed='left').sum()
        
    for i in rolling_back:
        data['prev_neighb_%dh'%i] = data.iloc[:, 1].rolling(i, i, closed='left').sum()
    
    for i in rolling_back:
        data['prev_dropoff_perc_%dh'%i] = data.iloc[:, 2].rolling(i, i, closed='left').sum()
    
    data.drop(['count', 'neighbors_sum', 'dropoff_percent'], axis=1, inplace=True)
    data.index.name = 'tpep_pickup_datetime'
    
    return data



# гармоники Фурье
def fourier_features(data, k=15):
    for i in range(1,k+1):
        data['sd%d'%i] = np.sin(np.arange(len(data)) * 2 * np.pi * i / 24)
        data['cd%d'%i] = np.cos(np.arange(len(data)) * 2 * np.pi * i / 24)
        data['sw%d'%i] = np.sin(np.arange(len(data)) * 2 * np.pi * i / 168)
        data['cw%d'%i] = np.cos(np.arange(len(data)) * 2 * np.pi * i / 168)
        
    return data



# удаление высококоррелированных между собой признаков
def hight_correlated_features(data, t):
    corr = data.corr()
    corrs = []
    
    for i, col in enumerate(corr.columns):
        for j, row in enumerate(corr.index):
            if j > i:
                corrs.append([col, row, corr.loc[row, col]])
    corrs = pd.DataFrame([[col,row,corr.loc[row, col],abs(corr.loc[row,col])]
                          for i,col in enumerate(corr.columns)for j,row in enumerate(corr.index) if j > i])
    corrs.columns = ['feat1', 'feat2', 'corr', 'abs_corr']
    corrs.sort_values(by='abs_corr', ascending=False, inplace=True)
    
    to_drop_cols = corrs[corrs['abs_corr'] >= t]['feat2'].values
    
    return to_drop_cols



# подбор коэф. альфа для Lasso
def alpha_search(X_train, y_train, X_test, y_test, alphas, estimator, tol=1e-1, max_iter=1e3):
    mae_ = []
    for alpha in alphas:
        estimator.alpha = alpha
        estimator.tol=tol
        estimator.max_iter=max_iter
        estimator.fit(X_train, y_train)
        pred = estimator.predict(X_test)
        mae = mean_absolute_error(y_test, pred)
        mae_.append(mae)

    return alphas[np.array(mae_).argmin()]

In [5]:
month_list = [[2016, i] for i in range(1, 7)]
folder_path = r'/Users/salavat/OneDrive/3. Learn/ML_DA/COURSE6/Прогнозирование временных рядов/DATA/NYC_TLC/'
file_list = [folder_path + f'yellow_tripdata_{year}-{month:02}.csv' for year, month in month_list]

In [6]:
# загружаем и обрабатываем данные
data_raw, data = load_transform_pipeline(file_list)

data_raw.to_csv('data_raw.csv')
data.to_csv('data.csv')

data = pd.read_csv('data.csv', parse_dates=[0])
data = data.set_index('tpep_pickup_datetime')

file 1 of 6 processing
file 2 of 6 processing
file 3 of 6 processing
file 4 of 6 processing
file 5 of 6 processing
file 6 of 6 processing


In [7]:
# составим сводную по количеству поездок в каждом регионе
regions_count = data.pivot_table(values = 'count', index = 'tpep_pickup_datetime',
                                 columns = 'pickup_region', fill_value = 0)
regions_count.to_csv('regions_count.csv')

regions_count = pd.read_csv('regions_count.csv', parse_dates=[0])
regions_count = regions_count.set_index('tpep_pickup_datetime')

In [8]:
# посчитаем долю каждого региона в который совершались поездки
dropoff_region_count = data_raw.groupby(['tpep_dropoff_datetime', 'dropoff_region']).size().to_frame(name='count')
dropoff_region_pivot = dropoff_region_count.pivot_table(values = 'count',
                                                        index = 'tpep_dropoff_datetime',
                                                        columns = 'dropoff_region',
                                                        fill_value=0)
dropoff_region_percent = dropoff_region_pivot.apply(lambda x: x / sum(x) * 100, axis=1, result_type='expand')
dropoff_region_percent[regions].head(2)
del data_raw

In [9]:
# создадим отдельный датафрейм для каждого из 102 регионов
regions_dfs = []

for reg in regions:
    df = data[data['pickup_region'] == reg].drop('pickup_region', axis=1)
    df = pd.DataFrame(index = period_full).join(df)
    df.index.name = 'tpep_pickup_datetime'
    
    # добавим долю региона в который совершалась поездка
    df = df.join(dropoff_region_percent[[reg]].rename({reg: 'dropoff_percent'}, axis=1))
    
    # кол-во поездок в соседних регионах
    neighbour_regions = list(set([reg-1, reg+1, reg-50, reg+50]) & set(regions_count))
    df['neighbors_sum'] = regions_count[neighbour_regions].sum(axis=1)
    
    # временные признаки
    cols = ['count', 'neighbors_sum', 'dropoff_percent']
    df = df.join(time_features(df[cols].copy()))
    
    # удалим признаки cols характеризуемые текущим временем (кроме count)
    df.drop(cols[1:], axis=1, inplace=True)
    
    # гармоники Фурье
    df = fourier_features(df)
    
    # праздники
    df['index_day'] = df.index.floor('D')
    df = df.merge(holidays, how='left', left_on='index_day', right_index=True).fillna(0)    
    df.drop('index_day', axis=1, inplace=True)
    
    
    # отмасштабируем признаки
    scaler_train_period = pd.date_range(datetime(2015,3,1),datetime(2016,5,31,23), freq='H')
    scaler = MinMaxScaler().fit(df.loc[period_full, df.columns[1:]])
    df.iloc[:, 1:].values[:] = scaler.transform(df.loc[period_full, df.columns[1:]])
    
    # удалим сильно коррелированные между собой признаки
    to_drop = hight_correlated_features(df.loc[period_train, df.columns[1:]], 0.95)
    df.drop(to_drop, axis=1, inplace=True)
    
    # сдвинем целевую переменную на [1..6]ч для каждой 6-ти моделей регрессий и удалим признак <count>
    for i in range(1,7):
        df['count_next_%dh'%i] = df['count'].shift(-i)
    
    regions_dfs.append(df)

In [10]:
# подберем параметр alpha для Lasso
alpha_list = list(np.arange(0.1, 1, 0.1)) + list(range(1, 10))

alphas = [] # лучшие альфа для каждого региона
Q_may = [] # ошибки за май

for reg_pos in range(102):
    for i in range(1,7):
        df = regions_dfs[reg_pos].copy()
        X_train = df.loc[period_train, df.columns[1:-6]]
        y_train = df.loc[period_train, 'count_next_%dh'%i]
        
        X_validation = df.loc[period_validation, df.columns[1:-6]]
        y_validation = df.loc[period_validation, 'count_next_%dh'%i]
        
        best_alpha = alpha_search(X_train, y_train, X_validation, y_validation, alpha_list, Lasso())
        alphas.append(best_alpha)
        
        model = Lasso(alpha=best_alpha).fit(X_train, y_train)
        pred = model.predict(X_validation)
        Q_may += [abs(err) for err in pred - y_validation]

print('Ошибка за май MAE:', np.array(Q_may).mean())

Ошибка за май MAE: 24.059171980354666


In [11]:
Q_june = [] # ошибки за июнь
models = [] # модели регрессии (102 региона x 6 моделей на каждый час прогноза)

for reg_pos in range(102):
    for i in range(1,7):
        df = regions_dfs[reg_pos].copy()
        X_train = df.loc[period_train2, df.columns[1:-6]]
        y_train = df.loc[period_train2, 'count_next_%dh'%i]
        
        X_test = df.loc[period_test, df.columns[1:-6]]
        y_test = df.loc[period_test, 'count_next_%dh'%i]
        
        model = Lasso(alpha=alphas[reg_pos], max_iter=1e4).fit(X_train, y_train)
        models.append(model)
        pred = model.predict(X_test)
        Q_june += [abs(err) for err in pred - y_test]
        
        
print('Ошибка за июнь MAE:', np.mean(Q_june))

Ошибка за июнь MAE: 22.772736957684018


In [12]:
# посмотрим на коэфиценты регрессии одного из регионов
coef_df = pd.DataFrame(models[70*7].coef_, index = regions_dfs[70].columns[1:-6], columns = ['coef'])
coef_df.index.name = 'feature'
coef_df['abs_coef'] = coef_df['coef'].apply(abs)
coef_df.sort_values(by='abs_coef', ascending=False, inplace=True)
coef_df.head(10)

,coef,abs_coef
feature,,
sd2,3.995338,3.995338
sd3,3.096218,3.096218
sd1,2.618178,2.618178
sd4,2.152269,2.152269
cd5,-1.806796,1.806796
cd2,1.275179,1.275179
sw8,-1.192878,1.192878
cw15,0.864279,0.864279
sw13,0.744913,0.744913


### Видно, что наибольшие коэфиценты получили признаки гармоник Фурье.<br>Дополнительные признаки не сильно помогли.<br>Думаю улучить результат помог бы учет годовой сезонности.